# 🏔️ Gradiente Descendente 1: A Descida na Direção Certa

## Pedro Nunes Guth - Cálculo para IA (Módulo 11/12)

Bora galera! Chegou a hora de colocar a mão na massa e implementar o algoritmo mais famoso do machine learning! Se você chegou até aqui, já estudou derivadas, gradientes e derivadas parciais. Agora é hora de usar tudo isso para fazer o que realmente importa: **ensinar uma máquina a aprender!**

Imagina que você tá perdido na Serra da Mantiqueira de noite e precisa descer da montanha. Como você faria? Óbvio: sentiria com o pé onde tá mais íngreme pra baixo e daria um passo nessa direção. É exatamente isso que o Gradiente Descendente faz!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-11_img_01.png)

In [ ]:
# Setup inicial - Bora importar as bibliotecas que vamos usar!
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Configurações para deixar os gráficos bonitos
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("🚀 Tudo pronto para a descida! Bora descer essa montanha do erro!")

## 🤔 Tá, mas o que é Gradiente Descendente afinal?

Lembra dos módulos anteriores? A gente viu que:
- **Derivadas** nos dão a inclinação de uma função em um ponto
- **Gradientes** apontam na direção de **maior crescimento** de uma função
- **Funções de custo** representam o "erro" do nosso modelo

O Gradiente Descendente é o algoritmo que usa essas informações para **minimizar** uma função de custo. A ideia é simples:

1. **Calcule o gradiente** (a direção de maior crescimento)
2. **Vá na direção oposta** (para diminuir o erro)
3. **Repita** até chegar no mínimo

### A Matemática por Trás (Descomplicada!)

Se temos uma função de custo $J(\theta)$ que queremos minimizar, o algoritmo funciona assim:

$$\theta_{novo} = \theta_{atual} - \alpha \cdot \nabla J(\theta_{atual})$$

Onde:
- $\theta$ são os **parâmetros** do modelo (pesos, bias, etc.)
- $\alpha$ é a **taxa de aprendizado** (o tamanho do passo)
- $\nabla J(\theta)$ é o **gradiente** da função de custo

**Dica do Pedro**: O sinal negativo é o pulo do gato! O gradiente aponta "pra cima", mas queremos ir "pra baixo" para minimizar o erro!

## 🎯 Visualizando o Conceito: A Montanha do Erro

Antes de codar, vamos visualizar o que tá rolando. Imagina uma função de custo simples como uma montanha:

In [ ]:
# Vamos criar uma função de custo simples: uma parábola
def funcao_custo(x):
    """Função de custo simples: (x - 3)² + 1
    O mínimo global está em x = 3
    """
    return (x - 3)**2 + 1

def derivada_custo(x):
    """Derivada da função de custo: 2(x - 3)
    Nos dá a inclinação em qualquer ponto x
    """
    return 2 * (x - 3)

# Criando os pontos para o gráfico
x = np.linspace(-2, 8, 1000)
y = funcao_custo(x)

# Plotando a "montanha do erro"
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(x, y, 'b-', linewidth=3, label='Função de Custo J(x) = (x-3)² + 1')
ax.axvline(x=3, color='r', linestyle='--', alpha=0.7, label='Mínimo Global (x=3)')
ax.scatter([3], [1], color='red', s=200, zorder=5, label='Ponto Ótimo')

# Marcando alguns pontos de exemplo
pontos_exemplo = [0, 1.5, 4.5, 6]
for ponto in pontos_exemplo:
    y_ponto = funcao_custo(ponto)
    inclinacao = derivada_custo(ponto)
    
    # Plotando o ponto
    ax.scatter([ponto], [y_ponto], color='orange', s=100, zorder=4)
    
    # Plotando a reta tangente (mostra a direção do gradiente)
    dx = 0.5
    x_tangente = np.array([ponto - dx, ponto + dx])
    y_tangente = y_ponto + inclinacao * (x_tangente - ponto)
    ax.plot(x_tangente, y_tangente, 'g--', alpha=0.6, linewidth=2)
    
    # Seta mostrando a direção de descida (oposta ao gradiente)
    direcao = -np.sign(inclinacao) * 0.3
    ax.arrow(ponto, y_ponto + 0.5, direcao, 0, head_width=0.3, 
             head_length=0.1, fc='purple', ec='purple', linewidth=2)

ax.set_xlabel('Parâmetro θ', fontsize=14)
ax.set_ylabel('Custo J(θ)', fontsize=14)
ax.set_title('🏔️ A Montanha do Erro - Como o Gradiente Descendente Funciona', fontsize=16)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎯 Olha só! As setas roxas mostram a direção que devemos seguir para minimizar o erro!")
print("📐 As linhas verdes são as retas tangentes - elas mostram a inclinação (gradiente) em cada ponto.")

## 🔥 Implementando o Gradiente Descendente na Mão!

Agora vem a parte mais legal! Vamos implementar o algoritmo passo a passo. Sem bibliotecas prontas - só matemática pura!

### Algoritmo Passo a Passo:

```mermaid
graph TD
    A[Inicializar θ aleatoriamente] --> B[Calcular J(θ)]
    B --> C[Calcular ∇J(θ)]
    C --> D[Atualizar: θ = θ - α∇J(θ)]
    D --> E{Convergiu?}
    E -->|Não| B
    E -->|Sim| F[θ ótimo encontrado!]
```

In [ ]:
def gradiente_descendente(funcao_custo, derivada, theta_inicial, taxa_aprendizado, max_iteracoes=1000, tolerancia=1e-6):
    """
    Implementação do Gradiente Descendente do zero!
    
    Parâmetros:
    - funcao_custo: função que queremos minimizar
    - derivada: derivada da função de custo
    - theta_inicial: ponto de partida
    - taxa_aprendizado: tamanho do passo (α)
    - max_iteracoes: número máximo de iterações
    - tolerancia: critério de parada
    
    Retorna:
    - theta_otimo: parâmetro que minimiza a função
    - historico: lista com todos os valores durante a otimização
    """
    
    # Inicializando
    theta = theta_inicial
    historico_theta = [theta]
    historico_custo = [funcao_custo(theta)]
    historico_gradiente = [derivada(theta)]
    
    print(f"🚀 Começando a descida do ponto θ = {theta:.4f}")
    print(f"📊 Custo inicial: {funcao_custo(theta):.4f}")
    print(f"📈 Gradiente inicial: {derivada(theta):.4f}")
    print("="*50)
    
    for i in range(max_iteracoes):
        # 1. Calcular o gradiente no ponto atual
        gradiente_atual = derivada(theta)
        
        # 2. Atualizar theta na direção oposta ao gradiente
        theta_novo = theta - taxa_aprendizado * gradiente_atual
        
        # 3. Salvar no histórico
        historico_theta.append(theta_novo)
        historico_custo.append(funcao_custo(theta_novo))
        historico_gradiente.append(derivada(theta_novo))
        
        # 4. Verificar convergência
        if abs(theta_novo - theta) < tolerancia:
            print(f"✅ Convergiu na iteração {i+1}!")
            break
            
        # 5. Mostrar progresso a cada 100 iterações
        if (i + 1) % 100 == 0:
            print(f"Iteração {i+1}: θ = {theta_novo:.6f}, Custo = {funcao_custo(theta_novo):.6f}")
        
        # 6. Atualizar theta para a próxima iteração
        theta = theta_novo
    
    print("="*50)
    print(f"🎯 Resultado final:")
    print(f"   θ ótimo = {theta:.6f}")
    print(f"   Custo mínimo = {funcao_custo(theta):.6f}")
    print(f"   Gradiente final = {derivada(theta):.6f}")
    
    return theta, {
        'theta': historico_theta,
        'custo': historico_custo,
        'gradiente': historico_gradiente
    }

print("🛠️ Função do Gradiente Descendente implementada! Agora vamos testá-la!")

## 🧪 Testando o Algoritmo: Primeira Descida!

Bora testar nosso gradiente descendente com a função que criamos: $J(\theta) = (\theta - 3)^2 + 1$

Sabemos que o mínimo tá em $\theta = 3$. Será que nosso algoritmo vai conseguir encontrar?

**Dica do Pedro**: A taxa de aprendizado é crucial! Muito alta e o algoritmo vai "pular" o mínimo, muito baixa e vai demorar uma eternidade para convergir.

In [ ]:
# Teste 1: Taxa de aprendizado boa (0.1)
print("🎯 TESTE 1: Taxa de aprendizado = 0.1")
theta_otimo_1, historico_1 = gradiente_descendente(
    funcao_custo=funcao_custo,
    derivada=derivada_custo, 
    theta_inicial=0.0,  # Começando longe do mínimo
    taxa_aprendizado=0.1,
    max_iteracoes=1000
)

print(f"\n📊 Número de iterações: {len(historico_1['theta']) - 1}")
print(f"🎯 Erro do resultado: {abs(theta_otimo_1 - 3):.8f}")

In [ ]:
# Vamos visualizar a descida do gradiente!
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Trajetória na função de custo
x_plot = np.linspace(-1, 7, 1000)
y_plot = funcao_custo(x_plot)

ax1.plot(x_plot, y_plot, 'b-', linewidth=2, label='J(θ) = (θ-3)² + 1')
ax1.plot(historico_1['theta'], historico_1['custo'], 'ro-', markersize=4, 
         linewidth=2, alpha=0.7, label='Trajetória do GD')
ax1.scatter([historico_1['theta'][0]], [historico_1['custo'][0]], 
           color='green', s=150, marker='s', label='Início', zorder=5)
ax1.scatter([historico_1['theta'][-1]], [historico_1['custo'][-1]], 
           color='red', s=150, marker='*', label='Final', zorder=5)
ax1.set_xlabel('θ')
ax1.set_ylabel('Custo J(θ)')
ax1.set_title('🏔️ Descida na Montanha do Erro')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Evolução do parâmetro θ
iteracoes = range(len(historico_1['theta']))
ax2.plot(iteracoes, historico_1['theta'], 'g-', linewidth=2, marker='o', markersize=3)
ax2.axhline(y=3, color='r', linestyle='--', alpha=0.7, label='θ ótimo = 3')
ax2.set_xlabel('Iteração')
ax2.set_ylabel('θ')
ax2.set_title('📈 Convergência do Parâmetro')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Evolução do custo
ax3.plot(iteracoes, historico_1['custo'], 'purple', linewidth=2, marker='o', markersize=3)
ax3.axhline(y=1, color='r', linestyle='--', alpha=0.7, label='Custo mínimo = 1')
ax3.set_xlabel('Iteração')
ax3.set_ylabel('Custo J(θ)')
ax3.set_title('📉 Redução do Custo')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Gráfico 4: Evolução do gradiente
ax4.plot(iteracoes, historico_1['gradiente'], 'orange', linewidth=2, marker='o', markersize=3)
ax4.axhline(y=0, color='r', linestyle='--', alpha=0.7, label='Gradiente = 0')
ax4.set_xlabel('Iteração')
ax4.set_ylabel('Gradiente ∇J(θ)')
ax4.set_title('🧭 Convergência do Gradiente')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎉 Liiindo! Olha como o algoritmo convergiu perfeitamente!")
print("🔍 Repare que quando o gradiente chega perto de zero, encontramos o mínimo!")

## ⚡ O Papel Crucial da Taxa de Aprendizado

A taxa de aprendizado ($\alpha$) é como a velocidade que você desce a montanha. Muito devagar e você nunca chega, muito rápido e você pode passar direto pelo vale!

Vamos comparar diferentes taxas de aprendizado:

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-11_img_02.png)

In [ ]:
# Testando diferentes taxas de aprendizado
taxas = [0.01, 0.1, 0.5, 1.0]
cores = ['blue', 'green', 'orange', 'red']
resultados = {}

print("🧪 Testando diferentes taxas de aprendizado...\n")

for i, taxa in enumerate(taxas):
    print(f"📊 Taxa de aprendizado: {taxa}")
    print("="*30)
    
    try:
        theta_opt, hist = gradiente_descendente(
            funcao_custo=funcao_custo,
            derivada=derivada_custo,
            theta_inicial=0.0,
            taxa_aprendizado=taxa,
            max_iteracoes=100  # Limitando para comparar
        )
        resultados[taxa] = hist
    except:
        print(f"❌ Taxa {taxa} causou instabilidade!")
    
    print("\n")

In [ ]:
# Visualizando o impacto das diferentes taxas
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Convergência do parâmetro θ
for i, (taxa, hist) in enumerate(resultados.items()):
    iteracoes = range(len(hist['theta']))
    ax1.plot(iteracoes, hist['theta'], color=cores[i], linewidth=2, 
             marker='o', markersize=3, label=f'α = {taxa}')

ax1.axhline(y=3, color='black', linestyle='--', alpha=0.7, label='θ ótimo = 3')
ax1.set_xlabel('Iteração')
ax1.set_ylabel('θ')
ax1.set_title('🎯 Convergência com Diferentes Taxas de Aprendizado')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-1, 7)

# Gráfico 2: Evolução do custo
for i, (taxa, hist) in enumerate(resultados.items()):
    iteracoes = range(len(hist['custo']))
    ax2.plot(iteracoes, hist['custo'], color=cores[i], linewidth=2, 
             marker='o', markersize=3, label=f'α = {taxa}')

ax2.axhline(y=1, color='black', linestyle='--', alpha=0.7, label='Custo mínimo = 1')
ax2.set_xlabel('Iteração')
ax2.set_ylabel('Custo J(θ)')
ax2.set_title('📉 Redução do Custo com Diferentes Taxas')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')  # Escala logarítmica para ver melhor

plt.tight_layout()
plt.show()

print("🔍 Análise dos resultados:")
print("🐌 α = 0.01: Muito devagar, mas estável")
print("✅ α = 0.1: Velocidade boa, convergência suave")
print("⚡ α = 0.5: Mais rápido, mas pode oscilar")
print("💥 α = 1.0: Muito rápido, pode causar instabilidade")

print("\n**Dica do Pedro**: Na vida real, começamos com α ≈ 0.01 a 0.1 e ajustamos conforme necessário!")

## 🌍 Exemplo Prático: Regressão Linear com Gradiente Descendente

Agora vamos aplicar o gradiente descendente em um problema real: **Regressão Linear!** 

Lembra da equação da reta? $y = mx + b$

No machine learning, escrevemos como: $\hat{y} = \theta_0 + \theta_1 x$

Onde:
- $\theta_0$ é o intercepto (bias)
- $\theta_1$ é a inclinação (peso)

### A Função de Custo: Erro Quadrático Médio (MSE)

$$J(\theta_0, \theta_1) = \frac{1}{2m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})^2$$

$$J(\theta_0, \theta_1) = \frac{1}{2m} \sum_{i=1}^{m} (\theta_0 + \theta_1 x^{(i)} - y^{(i)})^2$$

### As Derivadas Parciais:

$$\frac{\partial J}{\partial \theta_0} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)})$$

$$\frac{\partial J}{\partial \theta_1} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}^{(i)} - y^{(i)}) \cdot x^{(i)}$$

**Dica do Pedro**: Essas derivadas parciais nos dizem como cada parâmetro afeta o erro total!

In [ ]:
# Gerando dados sintéticos para regressão linear
np.random.seed(42)  # Para resultados reproduzíveis

# Dados verdadeiros: y = 2x + 1 + ruído
n_amostras = 100
X = np.random.uniform(-3, 3, n_amostras)
y_verdadeiro = 2 * X + 1  # Linha verdadeira
ruido = np.random.normal(0, 0.5, n_amostras)
y = y_verdadeiro + ruido

print(f"📊 Dataset criado:")
print(f"   {n_amostras} amostras")
print(f"   Relação verdadeira: y = 2x + 1")
print(f"   Ruído adicionado para realismo")

# Visualizando os dados
fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(X, y, alpha=0.6, color='blue', s=50, label='Dados observados')
ax.plot(X, y_verdadeiro, 'r--', linewidth=2, label='Relação verdadeira: y = 2x + 1')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('📈 Dataset para Regressão Linear')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

print("🎯 Objetivo: Usar gradiente descendente para encontrar θ₀ ≈ 1 e θ₁ ≈ 2")

In [ ]:
class RegressaoLinearGD:
    """Regressão Linear implementada com Gradiente Descendente do zero!"""
    
    def __init__(self, taxa_aprendizado=0.01, max_iteracoes=1000, tolerancia=1e-6):
        self.taxa_aprendizado = taxa_aprendizado
        self.max_iteracoes = max_iteracoes
        self.tolerancia = tolerancia
        self.historico = {'custo': [], 'theta0': [], 'theta1': []}
        
    def funcao_custo(self, X, y, theta0, theta1):
        """Calcula o Erro Quadrático Médio (MSE)"""
        m = len(X)
        predicoes = theta0 + theta1 * X
        custo = (1/(2*m)) * np.sum((predicoes - y)**2)
        return custo
    
    def calcular_gradientes(self, X, y, theta0, theta1):
        """Calcula as derivadas parciais"""
        m = len(X)
        predicoes = theta0 + theta1 * X
        erros = predicoes - y
        
        # Derivadas parciais
        d_theta0 = (1/m) * np.sum(erros)
        d_theta1 = (1/m) * np.sum(erros * X)
        
        return d_theta0, d_theta1
    
    def fit(self, X, y):
        """Treina o modelo usando Gradiente Descendente"""
        # Inicialização aleatória dos parâmetros
        self.theta0 = np.random.normal(0, 0.1)
        self.theta1 = np.random.normal(0, 0.1)
        
        print(f"🚀 Iniciando treinamento...")
        print(f"   Parâmetros iniciais: θ₀ = {self.theta0:.4f}, θ₁ = {self.theta1:.4f}")
        print(f"   Taxa de aprendizado: {self.taxa_aprendizado}")
        print("="*60)
        
        for i in range(self.max_iteracoes):
            # Calcular custo atual
            custo_atual = self.funcao_custo(X, y, self.theta0, self.theta1)
            
            # Calcular gradientes
            d_theta0, d_theta1 = self.calcular_gradientes(X, y, self.theta0, self.theta1)
            
            # Salvar no histórico
            self.historico['custo'].append(custo_atual)
            self.historico['theta0'].append(self.theta0)
            self.historico['theta1'].append(self.theta1)
            
            # Atualizar parâmetros (GRADIENTE DESCENDENTE!)
            theta0_novo = self.theta0 - self.taxa_aprendizado * d_theta0
            theta1_novo = self.theta1 - self.taxa_aprendizado * d_theta1
            
            # Verificar convergência
            if abs(theta0_novo - self.theta0) < self.tolerancia and abs(theta1_novo - self.theta1) < self.tolerancia:
                print(f"✅ Convergência atingida na iteração {i+1}!")
                break
                
            # Mostrar progresso
            if (i + 1) % 100 == 0:
                print(f"Iteração {i+1}: Custo = {custo_atual:.6f}, θ₀ = {theta0_novo:.4f}, θ₁ = {theta1_novo:.4f}")
            
            # Atualizar parâmetros
            self.theta0 = theta0_novo
            self.theta1 = theta1_novo
        
        print("="*60)
        print(f"🎯 Treinamento concluído!")
        print(f"   θ₀ (intercepto) = {self.theta0:.6f} (verdadeiro: 1.0)")
        print(f"   θ₁ (inclinação) = {self.theta1:.6f} (verdadeiro: 2.0)")
        print(f"   Custo final = {self.historico['custo'][-1]:.6f}")
        
    def predict(self, X):
        """Faz predições"""
        return self.theta0 + self.theta1 * X

# Criando e treinando o modelo
modelo = RegressaoLinearGD(taxa_aprendizado=0.1, max_iteracoes=1000)
modelo.fit(X, y)

## 📊 Visualizando o Aprendizado em Ação

Agora vem a parte mais legal: ver como o modelo **aprende** durante o treinamento!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-11_img_03.png)

In [ ]:
# Visualizando o processo de aprendizado
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Gráfico 1: Evolução da linha de regressão
ax1.scatter(X, y, alpha=0.6, color='lightblue', s=30, label='Dados')
ax1.plot(X, y_verdadeiro, 'r--', linewidth=2, label='y = 2x + 1 (verdadeiro)')

# Mostrar algumas iterações do aprendizado
X_plot = np.linspace(X.min(), X.max(), 100)
iteracoes_mostrar = [0, len(modelo.historico['theta0'])//4, len(modelo.historico['theta0'])//2, -1]
cores_linha = ['red', 'orange', 'green', 'blue']
labels_linha = ['Início', '25%', '50%', 'Final']

for i, (iter_idx, cor, label) in enumerate(zip(iteracoes_mostrar, cores_linha, labels_linha)):
    theta0_iter = modelo.historico['theta0'][iter_idx]
    theta1_iter = modelo.historico['theta1'][iter_idx]
    y_pred_iter = theta0_iter + theta1_iter * X_plot
    ax1.plot(X_plot, y_pred_iter, color=cor, linewidth=2, alpha=0.8, label=f'{label}: y = {theta1_iter:.2f}x + {theta0_iter:.2f}')

ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('🎯 Evolução da Linha de Regressão')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Convergência do custo
iteracoes = range(len(modelo.historico['custo']))
ax2.plot(iteracoes, modelo.historico['custo'], 'purple', linewidth=2)
ax2.set_xlabel('Iteração')
ax2.set_ylabel('Custo (MSE)')
ax2.set_title('📉 Redução do Custo Durante o Treinamento')
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

# Gráfico 3: Convergência dos parâmetros
ax3.plot(iteracoes, modelo.historico['theta0'], 'blue', linewidth=2, label='θ₀ (intercepto)')
ax3.plot(iteracoes, modelo.historico['theta1'], 'red', linewidth=2, label='θ₁ (inclinação)')
ax3.axhline(y=1, color='blue', linestyle='--', alpha=0.7, label='θ₀ verdadeiro = 1')
ax3.axhline(y=2, color='red', linestyle='--', alpha=0.7, label='θ₁ verdadeiro = 2')
ax3.set_xlabel('Iteração')
ax3.set_ylabel('Valor do Parâmetro')
ax3.set_title('📈 Convergência dos Parâmetros')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Gráfico 4: Superfície de custo 3D (conceitual)
theta0_range = np.linspace(0, 2, 50)
theta1_range = np.linspace(1, 3, 50)
THETA0, THETA1 = np.meshgrid(theta0_range, theta1_range)

# Calculando a superfície de custo
CUSTO = np.zeros_like(THETA0)
for i in range(THETA0.shape[0]):
    for j in range(THETA0.shape[1]):
        CUSTO[i, j] = modelo.funcao_custo(X, y, THETA0[i, j], THETA1[i, j])

contour = ax4.contour(THETA0, THETA1, CUSTO, levels=20, alpha=0.6)
ax4.clabel(contour, inline=True, fontsize=8)

# Plotando a trajetória do gradiente descendente
ax4.plot(modelo.historico['theta0'], modelo.historico['theta1'], 'ro-', 
         linewidth=2, markersize=3, alpha=0.8, label='Trajetória GD')
ax4.scatter([modelo.historico['theta0'][0]], [modelo.historico['theta1'][0]], 
           color='green', s=100, marker='s', label='Início', zorder=5)
ax4.scatter([modelo.historico['theta0'][-1]], [modelo.historico['theta1'][-1]], 
           color='red', s=150, marker='*', label='Final', zorder=5)
ax4.scatter([1], [2], color='blue', s=100, marker='x', label='Ótimo verdadeiro', zorder=5)

ax4.set_xlabel('θ₀ (intercepto)')
ax4.set_ylabel('θ₁ (inclinação)')
ax4.set_title('🗺️ Trajetória na Superfície de Custo')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🎉 Liiindo! Olha como o gradiente descendente encontrou os parâmetros quase perfeitamente!")
print(f"💡 Erro no θ₀: {abs(modelo.theta0 - 1):.6f}")
print(f"💡 Erro no θ₁: {abs(modelo.theta1 - 2):.6f}")

## 🔥 Exercício Prático 1: Sua Vez de Implementar!

Agora é sua vez! Vou dar uma função de custo diferente e você vai implementar o gradiente descendente para ela.

**Função:** $J(\theta) = \theta^4 - 4\theta^3 + 6\theta^2 - 4\theta + 5$

**Derivada:** $J'(\theta) = 4\theta^3 - 12\theta^2 + 12\theta - 4$

**Desafio:** Encontre o mínimo global desta função!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-11_img_04.png)

In [ ]:
# EXERCÍCIO 1: Complete as funções abaixo

def funcao_exercicio(theta):
    """TODO: Implemente J(θ) = θ⁴ - 4θ³ + 6θ² - 4θ + 5"""
    # Sua implementação aqui
    pass

def derivada_exercicio(theta):
    """TODO: Implemente J'(θ) = 4θ³ - 12θ² + 12θ - 4"""
    # Sua implementação aqui
    pass

# TODO: Use a função gradiente_descendente que criamos para encontrar o mínimo
# Teste com diferentes pontos iniciais: -1, 0, 2, 3
# Dica: Esta função tem múltiplos mínimos locais!

print("🎯 Seu desafio: encontrar o mínimo global da função!")
print("💡 Dica: Teste diferentes pontos iniciais para ver o que acontece!")

# Descomente as linhas abaixo quando implementar as funções
# theta_otimo, historico = gradiente_descendente(
#     funcao_custo=funcao_exercicio,
#     derivada=derivada_exercicio,
#     theta_inicial=0.0,  # Experimente outros valores!
#     taxa_aprendizado=0.01,
#     max_iteracoes=1000
# )

## 🚨 Problemas Comuns no Gradiente Descendente

Como toda técnica poderosa, o gradiente descendente tem suas pegadinhas. Vamos ver os principais problemas:

### 1. 🐌 Mínimos Locais vs Mínimo Global

Imagina que você tá descendo uma montanha no escuro e cai num buraco. Você pode pensar que chegou no fundo, mas na verdade existe um vale muito mais profundo do lado!

```mermaid
graph TD
    A[Função não-convexa] --> B{Múltiplos mínimos}
    B --> C[Mínimo Local 1]
    B --> D[Mínimo Local 2]
    B --> E[Mínimo Global]
    C --> F[❌ Pode parar aqui]
    D --> F
    E --> G[✅ Queremos chegar aqui]
```

### 2. 🎢 Taxa de Aprendizado Inadequada

- **Muito baixa**: Converge muito devagar (como uma lesma subindo no pau de sebo)
- **Muito alta**: Fica pulando de um lado pro outro sem nunca convergir (como um pintinho tonto)

### 3. 🏔️ Platôs e Selas

Às vezes o gradiente fica muito pequeno em regiões "planas", fazendo o algoritmo andar muito devagar.

**Dica do Pedro**: Por isso surgiram versões melhoradas como Adam, RMSprop, etc. (que veremos no próximo módulo!)

In [ ]:
# Demonstrando o problema dos mínimos locais
def funcao_multimodal(x):
    """Função com múltiplos mínimos locais"""
    return x**4 - 4*x**3 + 6*x**2 - 4*x + 5

def derivada_multimodal(x):
    """Derivada da função multimodal"""
    return 4*x**3 - 12*x**2 + 12*x - 4

# Testando diferentes pontos iniciais
pontos_iniciais = [-0.5, 0.5, 1.5, 2.5]
resultados_multimodal = {}

print("🧪 Testando o problema dos mínimos locais...\n")

for ponto in pontos_iniciais:
    print(f"📍 Ponto inicial: {ponto}")
    theta_opt, hist = gradiente_descendente(
        funcao_custo=funcao_multimodal,
        derivada=derivada_multimodal,
        theta_inicial=ponto,
        taxa_aprendizado=0.01,
        max_iteracoes=500
    )
    resultados_multimodal[ponto] = (theta_opt, hist)
    print(f"🎯 Convergiu para: θ = {theta_opt:.4f}, J(θ) = {funcao_multimodal(theta_opt):.4f}\n")

In [ ]:
# Visualizando o problema dos mínimos locais
x_plot = np.linspace(-1, 3, 1000)
y_plot = funcao_multimodal(x_plot)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Função com trajetórias
ax1.plot(x_plot, y_plot, 'b-', linewidth=3, label='J(θ) = θ⁴ - 4θ³ + 6θ² - 4θ + 5')

cores = ['red', 'green', 'orange', 'purple']
for i, (ponto_inicial, (theta_final, hist)) in enumerate(resultados_multimodal.items()):
    # Trajetória
    ax1.plot(hist['theta'], hist['custo'], 'o-', color=cores[i], 
             alpha=0.7, markersize=3, linewidth=2, 
             label=f'Início: {ponto_inicial} → Final: {theta_final:.2f}')
    
    # Ponto inicial
    ax1.scatter([ponto_inicial], [funcao_multimodal(ponto_inicial)], 
               color=cores[i], s=150, marker='s', zorder=5)
    
    # Ponto final
    ax1.scatter([theta_final], [funcao_multimodal(theta_final)], 
               color=cores[i], s=200, marker='*', zorder=5)

ax1.set_xlabel('θ')
ax1.set_ylabel('J(θ)')
ax1.set_title('🎢 Problema dos Mínimos Locais')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Convergência do custo
for i, (ponto_inicial, (theta_final, hist)) in enumerate(resultados_multimodal.items()):
    iteracoes = range(len(hist['custo']))
    ax2.plot(iteracoes, hist['custo'], color=cores[i], linewidth=2,
             label=f'Início: {ponto_inicial}')

ax2.set_xlabel('Iteração')
ax2.set_ylabel('Custo J(θ)')
ax2.set_title('📉 Convergência para Diferentes Mínimos')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

print("🔍 Análise dos resultados:")
print("• Pontos iniciais diferentes podem levar a mínimos diferentes!")
print("• Este é um dos grandes desafios da otimização não-convexa")
print("• Na prática: rodamos várias vezes com inicializações aleatórias")

print("\n**Dica do Pedro**: Em redes neurais, a inicialização dos pesos é super importante!")

## 🚀 Exercício Prático 2: Comparando Taxas de Aprendizado

Agora você vai experimentar com diferentes taxas de aprendizado e ver como elas afetam a convergência.

**Sua missão**: Testar as taxas [0.001, 0.01, 0.1, 0.5] na função do exercício anterior e comparar:
- Velocidade de convergência
- Estabilidade
- Qual chega mais perto do mínimo verdadeiro

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-11_img_05.png)

In [ ]:
# EXERCÍCIO 2: Complete o código para comparar taxas de aprendizado

# TODO: Teste estas taxas de aprendizado na função multimodal
taxas_teste = [0.001, 0.01, 0.1, 0.5]
ponto_inicial_fixo = 0.0

print("🏁 EXERCÍCIO: Comparando taxas de aprendizado")
print("="*50)

# TODO: Para cada taxa, execute o gradiente descendente e compare:
# 1. Número de iterações até convergir
# 2. Valor final do custo
# 3. Valor final do parâmetro θ
# 4. Estabilidade (oscilações?)

resultados_taxas = {}

for taxa in taxas_teste:
    print(f"\n🎯 Testando taxa = {taxa}")
    print("-" * 30)
    
    # TODO: Chame a função gradiente_descendente aqui
    # Salve os resultados em resultados_taxas[taxa]
    
    pass  # Remova esta linha quando implementar

# TODO: Crie visualizações comparando os resultados
# Dicas: 
# - Gráfico da convergência do custo para cada taxa
# - Gráfico da convergência do parâmetro θ
# - Tabela resumo com os resultados finais

print("\n📊 Análise: Qual taxa funcionou melhor e por quê?")

## 🎯 Quando Usar Gradiente Descendente?

O gradiente descendente é o coração do machine learning moderno! Ele é usado em:

### ✅ **Ideal para:**
- **Redes neurais** (backpropagation é gradiente descendente!)
- **Regressão linear/logística** com muitos dados
- **Deep learning** (única opção viável para milhões de parâmetros)
- **Funções diferenciáveis** que não têm solução analítica

### ❌ **Não é ideal para:**
- **Funções não-diferenciáveis** (óbvio!)
- **Espaços discretos** (não tem "direção" para descer)
- **Problemas pequenos** com solução analítica simples

### 🔥 **Versões Modernas:**
- **SGD** (Stochastic): Usa mini-batches dos dados
- **Adam**: Adapta a taxa de aprendizado automaticamente  
- **RMSprop**: Lida melhor com gradientes que variam muito
- **Momentum**: Adiciona "inércia" para passar por mínimos locais

**Dica do Pedro**: No próximo módulo vamos ver essas versões turbinasdas! Prepare-se!

## 🧠 Conectando com o Que Vem Pela Frente

O gradiente descendente que implementamos hoje é a versão "batch" - usa todos os dados de uma vez. Mas e se tivermos 1 milhão de exemplos? Vai ser muito lento!

No **Módulo 12** (último do curso!), vamos ver:

### 🚀 **Stochastic Gradient Descent (SGD)**
Em vez de usar todos os dados, usa apenas um exemplo por vez:

$$\theta = \theta - \alpha \nabla J^{(i)}(\theta)$$

### ⚡ **Mini-batch Gradient Descent**
Meio termo: usa pequenos grupos de dados:

$$\theta = \theta - \alpha \frac{1}{m_{batch}} \sum_{j=1}^{m_{batch}} \nabla J^{(j)}(\theta)$$

### 🧠 **Adam Optimizer**
O mais usado hoje em deep learning - adapta a taxa de aprendizado:

$$m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t$$
$$v_t = \beta_2 v_{t-1} + (1-\beta_2) g_t^2$$

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/cálculo-para-ia-modulo-11_img_06.png)

In [ ]:
# Prévia do que vem no próximo módulo: Mini-batch
def mini_batch_preview(X, y, batch_size=10):
    """Prévia de como funciona o mini-batch (veremos no próximo módulo)"""
    
    n_amostras = len(X)
    indices = np.arange(n_amostras)
    np.random.shuffle(indices)  # Embaralha os dados
    
    print(f"📊 Dataset completo: {n_amostras} amostras")
    print(f"🎯 Tamanho do mini-batch: {batch_size}")
    print(f"🔄 Número de mini-batches: {n_amostras // batch_size}")
    print("\nExemplo de mini-batches:")
    
    for i in range(0, min(n_amostras, 3 * batch_size), batch_size):
        batch_indices = indices[i:i+batch_size]
        X_batch = X[batch_indices]
        y_batch = y[batch_indices]
        
        print(f"  Mini-batch {i//batch_size + 1}: {len(X_batch)} amostras")
        print(f"    X: [{X_batch[0]:.2f}, {X_batch[1]:.2f}, ..., {X_batch[-1]:.2f}]")
        print(f"    y: [{y_batch[0]:.2f}, {y_batch[1]:.2f}, ..., {y_batch[-1]:.2f}]")
    
    print("\n💡 No próximo módulo: cada mini-batch fará uma atualização dos parâmetros!")
    print("🚀 Isso torna o treinamento muito mais rápido e eficiente!")

# Demonstração
mini_batch_preview(X, y, batch_size=10)

## 📚 Resumo do Módulo: O Que Aprendemos

Parabéns! 🎉 Você acabou de dominar um dos algoritmos mais importantes do machine learning!

### ✅ **O que rolou neste módulo:**

1. **🧭 Entendemos o conceito**: Gradiente descendente é como descer uma montanha seguindo a direção mais íngreme

2. **📐 A matemática**: $\theta_{novo} = \theta_{atual} - \alpha \nabla J(\theta)$

3. **💻 Implementamos do zero**: Sem bibliotecas, só matemática pura!

4. **🎯 Aplicamos na prática**: Regressão linear completa com gradiente descendente

5. **⚠️ Vimos as pegadinhas**: Mínimos locais, taxa de aprendizado, convergência

6. **📊 Visualizamos tudo**: Gráficos que mostram como o algoritmo aprende

### 🔑 **Conceitos-chave para levar:**

- O **gradiente aponta "pra cima"**, então vamos na **direção oposta** para minimizar
- A **taxa de aprendizado** ($\alpha$) controla o tamanho do passo
- **Inicialização** importa em funções não-convexas
- **Convergência** acontece quando o gradiente fica próximo de zero

### 🚀 **Conectando com o curso:**

- **Módulos 1-3**: Prepararam o terreno (funções, limites)
- **Módulos 4-6**: Derivadas e regra da cadeia (base do gradiente)
- **Módulos 9-10**: Múltiplas variáveis e gradientes (matemática por trás)
- **Módulo 11** (atual): Implementação prática do algoritmo
- **Módulo 12** (próximo): Versões avançadas (SGD, Adam, etc.)

**Dica do Pedro**: Agora você entende o que tá rolando "por baixo dos panos" quando treina qualquer modelo de ML! Isso é poder! 💪

### 🎯 **Para o próximo módulo:**

Prepare-se para ver como o gradiente descendente evoluiu para lidar com:
- **Big Data** (milhões de exemplos)
- **Deep Learning** (milhões de parâmetros)  
- **Convergência mais rápida** (otimizadores adaptativos)
- **Estabilidade numérica** (truques dos profissionais)

Bora para o último módulo! 🚀